In [4]:
import paramiko
import scp

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
# ssh.connect('10.18.19.43', port=2002, username='root', password='root')
ssh.connect('10.64.128.188', port=2002, username='root', password='root')

scp_client = scp.SCPClient(ssh.get_transport())
local_folder_path = './'

scp_client.get('/root/SC24/pytorch/mytest/partition_test/bns_2_node1_partition.log', local_folder_path, recursive=True)
scp_client.get('/root/SC24/pytorch/mytest/partition_test/bns_2_node2_partition.log', local_folder_path, recursive=True)
scp_client.get('/root/SC24/pytorch/mytest/partition_test/bns_4_node1_partition.log', local_folder_path, recursive=True)
scp_client.get('/root/SC24/pytorch/mytest/partition_test/bns_4_node2_partition.log', local_folder_path, recursive=True)
scp_client.get('/root/SC24/pytorch/mytest/partition_test/bns_4_node3_partition.log', local_folder_path, recursive=True)
scp_client.get('/root/SC24/pytorch/mytest/partition_test/bns_4_node4_partition.log', local_folder_path, recursive=True)

scp_client.get('/root/SC24/pytorch/mytest/partition_test/rcgnn_2_node3_partition.log', local_folder_path, recursive=True)
scp_client.get('/root/SC24/pytorch/mytest/partition_test/rcgnn_2_node4_partition.log', local_folder_path, recursive=True)
scp_client.get('/root/SC24/pytorch/mytest/partition_test/rcgnn_4_node1_partition.log', local_folder_path, recursive=True)
scp_client.get('/root/SC24/pytorch/mytest/partition_test/rcgnn_4_node2_partition.log', local_folder_path, recursive=True)
scp_client.get('/root/SC24/pytorch/mytest/partition_test/rcgnn_4_node3_partition.log', local_folder_path, recursive=True)
scp_client.get('/root/SC24/pytorch/mytest/partition_test/rcgnn_4_node4_partition.log', local_folder_path, recursive=True)


scp_client.close()
ssh.close()


In [ ]:
import os

# 获取当前目录下的所有txt文件
txt_files = [f for f in os.listdir('.') if f.endswith('.log')]

for txt_file in txt_files:
    with open(txt_file, 'r') as f:
        lines = f.readlines()

    # 仅保留包含"for_commute"的行
    lines = [line for line in lines if 'for_commute' in line]

    # 创建同名的csv文件
    csv_file = txt_file.replace('.log', '.csv')
    with open(csv_file, 'w') as f:
        for line in lines:
            # 直接写入行，不添加额外的逗号
            f.write(line)

In [ ]:
# 读取所有csv
# for_commute,bns,rank 0 [(7, 692), (6, 2119), (5, 1604), (4, 3215), (3, 6886), (2, 7027), (1, 9367)] am-8-metis-cut-trans 8 4
# 变为
# for_commute,bns,0,(7,692),(6,2119),(5,1604),(4,3215),(3,6886),(2,7027),(1,9367),am-8-metis-cut-trans,8,4

import re

csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]

for csv_file in csv_files:
    with open(csv_file, 'r') as f:
        lines = f.readlines()

    with open(csv_file, 'w') as f:
        for line in lines:
            # 替换
            line = line.replace('rank ', '')
            line = line.replace(' [(', ',(')
            line = line.replace(', ', ',')
            line = line.replace(')]', ')')
            line = line.replace(' ', ',')
            # (7,692)类型的数据，替换为(7 692) 用正则表达式匹配，中间的数字可能会变
            line = re.sub(r'\((\d+),(\d+)\)', r'(\1 \2)', line)
            # line = line.replace('(', '')
            # line = line.replace(')', '')
            # ),(替换为)(
            line = line.replace('),(', ')(')

            f.write(line)


# for_commute,bns,rank 4 [(8, 223), (7, 804), (6, 746), (5, 2325), (3, 127), (2, 219), (1, 127), (0, 36), (15, 170), (14, 86), (13, 51), (12, 194), (11, 58), (10, 93), (9, 175)] ogbn-arxiv-16-metis-cut-trans 16 8
# for_commute,bns,4,8,223,7,804,6,746,5,2325,3,127,2,219,1,127,0,36,15,170,14,86,13,51,12,194,11,58,10,93,9,175,ogbn-arxiv-16-metis-cut-trans,16,8
